In [81]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [82]:
# Code Generation
message = "1101"  # 4-bit binary message

nr_codewords = int(100)
bits_info = torch.randint(2, (nr_codewords, 4), dtype=torch.float)

print(bits_info)

tensor([[1., 1., 1., 1.],
        [0., 0., 0., 1.],
        [1., 1., 0., 1.],
        [0., 0., 1., 0.],
        [0., 1., 0., 1.],
        [1., 1., 0., 0.],
        [0., 1., 0., 1.],
        [1., 0., 0., 0.],
        [1., 0., 1., 0.],
        [1., 1., 0., 0.],
        [0., 0., 0., 0.],
        [1., 0., 1., 1.],
        [0., 1., 1., 0.],
        [0., 1., 0., 1.],
        [1., 0., 1., 1.],
        [0., 1., 0., 0.],
        [1., 1., 0., 1.],
        [1., 1., 0., 0.],
        [0., 0., 1., 1.],
        [0., 0., 1., 1.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 1.],
        [1., 1., 1., 1.],
        [0., 1., 1., 1.],
        [1., 1., 0., 1.],
        [1., 0., 1., 1.],
        [1., 1., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 1.],
        [1., 0., 0., 1.],
        [1., 0., 0., 0.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [0., 0., 1., 0.],
        [0.,

Hamming(7,4) Encoder

In [83]:
class hamming_encode(torch.nn.Module):
    def __init__(self):
        """
            Use Hamming(7,4) to encode the data.
    
        Args:
            data: data received from the Hamming(7,4) encoder(Tensor)
            generator matrix: generate the parity code
    
        Returns:
            encoded data: 4 bits original info with 3 parity code.
        """
        super(hamming_encode, self).__init__()

        # Define the generator matrix for Hamming(7,4)
        self.generator_matrix = torch.tensor([
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
            [1, 1, 0, 1],
            [1, 0, 1, 1],
            [0, 1, 1, 1],
        ], dtype=torch.float)

    def forward(self, input_data):
        # Ensure input_data has shape (batch_size, 4)
        assert input_data.size(1) == 4, "Input data must have 4 bits."

        # Perform matrix multiplication to encode the data
        encoded_data = torch.matmul(input_data, self.generator_matrix.t()) % 2

        return encoded_data

In [84]:
encoder = hamming_encode()
encoded_codeword = encoder(bits_info)
print(encoded_codeword)

tensor([[1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1., 1.],
        [1., 1., 0., 1., 1., 0., 0.],
        [0., 0., 1., 0., 0., 1., 1.],
        [0., 1., 0., 1., 0., 1., 0.],
        [1., 1., 0., 0., 0., 1., 1.],
        [0., 1., 0., 1., 0., 1., 0.],
        [1., 0., 0., 0., 1., 1., 0.],
        [1., 0., 1., 0., 1., 0., 1.],
        [1., 1., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 1., 0., 1., 0.],
        [0., 1., 1., 0., 1., 1., 0.],
        [0., 1., 0., 1., 0., 1., 0.],
        [1., 0., 1., 1., 0., 1., 0.],
        [0., 1., 0., 0., 1., 0., 1.],
        [1., 1., 0., 1., 1., 0., 0.],
        [1., 1., 0., 0., 0., 1., 1.],
        [0., 0., 1., 1., 1., 0., 0.],
        [0., 0., 1., 1., 1., 0., 0.],
        [0., 1., 0., 0., 1., 0., 1.],
        [0., 0., 0., 1., 1., 1., 1.],
        [0., 1., 1., 1., 0., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [1., 0., 0., 0., 1., 1., 0.],
        [1., 0., 0., 1., 0., 0., 1.],
        [1.,

BPSK Modulator + Noise

In [85]:
class bpsk_modulator(torch.nn.Module):
    def __init__(self):
        """
        Use BPSK to compress the data, which is easily to transmit.

    Args:
        data: data received from the Hamming(7,4) encoder(Tensor)
        symbol_rate: Symbol rate in Hz
        carrier_freq: Carrier frequency in Hz
        snr_dB: Signal-to-noise ratio in dB

    Returns:
        time:
        data: Tensor contain all data modulated and add noise
    """
        super(bpsk_modulator, self).__init__()
        
    def forward(self, data, symbol_rate, carrier_freq, snr_dB):
    
        for i in range(data.shape[0]):
            bits = data[i]
            bits = 2 * bits - 1
            # Time vector
            time = torch.arange(0, len(bits) / symbol_rate, 1 / symbol_rate)
        
            # Generate carrier signal
            carrier = torch.cos(2 * torch.pi * carrier_freq * time)
        
            # Modulate the signal
            modulated_signal = bits * carrier
        
            # Add Gaussian noise to the signal
            noise_power = torch.tensor(10**(-snr_dB / 10))
            noise = torch.sqrt(noise_power) * torch.randn(len(modulated_signal))
            noised_signal = modulated_signal + noise
            data[i] = noised_signal
    
       
        return time, data

In [124]:
data = encoded_codeword  # Binary data
symbol_rate = 15  # Symbol rate in Hz
carrier_freq = 100  # Carrier frequency in Hz
snr_dB = 15  # Signal-to-noise ratio in dB

# Modulate the signal
modulator = bpsk_modulator()
time, modulated_noise_signal = modulator(data, symbol_rate, carrier_freq, snr_dB)
print(modulated_noise_signal)

tensor([[-5.8707e-01,  1.0589e+00,  4.9386e-01,  2.3466e+00,  4.5113e-01,
          8.3033e-01,  7.1847e-01],
        [-1.4957e+01,  5.7028e-01,  8.8216e-02, -4.0215e-01,  8.3130e-01,
          1.0035e+00,  2.4489e+00],
        [ 8.3695e-01,  1.5934e+00,  2.8013e-01, -3.3695e-01,  5.7704e-01,
          2.9488e-01, -1.6669e+01],
        [-1.6128e+01, -9.3115e-02,  1.0055e+00, -1.4332e+01,  3.6723e-01,
          1.1845e+00,  1.2458e+00],
        [-1.7102e+01,  8.9016e-01,  2.0087e-01,  7.6900e-01, -2.1254e-01,
          1.3262e+00, -1.4730e+01],
        [ 2.7797e+00,  1.1972e+00,  4.7485e-01, -1.7162e+01, -8.5971e-01,
          1.1904e+00,  8.8423e-01],
        [-1.5526e+01,  8.3048e-01,  9.8220e-02, -1.6100e-01,  6.5813e-02,
          8.0116e-01, -1.3472e+01],
        [-2.9066e+00, -5.5559e-01, -4.0629e-01, -1.6731e+01,  1.1645e+00,
          1.2057e+00, -1.3916e+01],
        [ 2.6239e+00,  6.6594e-02,  1.3810e+00, -1.3826e+01,  6.0298e-01,
          6.5756e-02, -1.4232e+00],
        [ 

LLR Log-likelihood
y = s + n
Assuming that \( s \) is equally likely to be 0 or 1, and \( n \) is Gaussian with zero mean and variance \( N_0/2 \), where \( N_0 \) is the noise power spectral density.


In [125]:
def llr(signal, snr):
    """
    Calculate Log Likelihood Ratio (LLR) for a simple binary symmetric channel.

    Args:
        signal (torch.Tensor): Received signal.
        noise_std (float): Standard deviation of the noise.

    Returns:
        torch.Tensor: Log Likelihood Ratio (LLR) values.
    """
    # Assuming Binary Phase Shift Keying (BPSK) modulation
    noise_std = torch.sqrt(torch.tensor(10**(snr / 10)))
    # likelihood_0 = 2 * (signal - 1) / noise_std**2
    # likelihood_1 = 2 * (signal + 1) / noise_std**2

    # Calculate the LLR
    # llr_values = likelihood_0 / likelihood_1
    llr = 2 * signal / noise_std**2

    # return llr_values, llr
    return llr


In [126]:
# received_signal = torch.tensor([2.6811e-01, -7.7690e-01, -3.2614e-01,  1.6319e+00, -1.5100e-01, -3.3976e-01,  8.3065e-01])
# LLR values: tensor([0.0170, -0.0491, -0.0206,  0.1032, -0.0096, -0.0215,  0.0525])

snr_dB = 15

llr = llr(modulated_noise_signal, snr_dB)
print("LLR values:", llr)

LLR values: tensor([[-3.7130e-02,  6.6973e-02,  3.1234e-02,  1.4841e-01,  2.8532e-02,
          5.2515e-02,  4.5440e-02],
        [-9.4596e-01,  3.6068e-02,  5.5793e-03, -2.5434e-02,  5.2576e-02,
          6.3467e-02,  1.5488e-01],
        [ 5.2933e-02,  1.0078e-01,  1.7717e-02, -2.1310e-02,  3.6495e-02,
          1.8650e-02, -1.0543e+00],
        [-1.0200e+00, -5.8891e-03,  6.3592e-02, -9.0642e-01,  2.3226e-02,
          7.4914e-02,  7.8793e-02],
        [-1.0816e+00,  5.6298e-02,  1.2704e-02,  4.8636e-02, -1.3442e-02,
          8.3877e-02, -9.3161e-01],
        [ 1.7580e-01,  7.5717e-02,  3.0032e-02, -1.0854e+00, -5.4373e-02,
          7.5285e-02,  5.5924e-02],
        [-9.8192e-01,  5.2524e-02,  6.2120e-03, -1.0183e-02,  4.1624e-03,
          5.0670e-02, -8.5202e-01],
        [-1.8383e-01, -3.5139e-02, -2.5696e-02, -1.0581e+00,  7.3650e-02,
          7.6257e-02, -8.8015e-01],
        [ 1.6595e-01,  4.2118e-03,  8.7344e-02, -8.7446e-01,  3.8136e-02,
          4.1588e-03, -9.0012e-02]

LDPC Decoder

In [96]:
import numpy as np

def ldpc_decode(received_codeword, max_iterations=50):
    # LDPC Parity-check matrix (7,4) example
    H = np.array([
        [1, 0, 1, 1, 0, 0, 0],
        [0, 1, 0, 0, 1, 0, 1],
        [0, 0, 0, 1, 0, 1, 0]
    ])

    # LDPC Generator matrix (7,4) example
    G = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 1, 1],
        [1, 0, 1, 1],
        [1, 1, 0, 1]
    ])

    # Initialize the received codeword and the decoded message
    y = np.array(list(map(int, received_codeword)))
    x_hat = np.zeros(G.shape[1])

    for iteration in range(max_iterations):
        # Syndrome check
        syndrome = np.mod(np.dot(H, y), 2)

        if np.all(syndrome == 0):
            # If the syndrome is all zeros, the decoding is successful
            break

        # Compute the log-likelihood ratios (LLRs) using the sum-product algorithm
        llrs = np.zeros(G.shape[1])
        for j in range(G.shape[1]):
            connected_checks = np.nonzero(H[:, j])[0]
            llrs[j] = 2 * y[j] / np.tanh(np.sum(np.arctanh(np.tanh(y[i] / 2)) for i in connected_checks))

        # Make hard decisions based on LLRs
        x_hat = np.where(llrs > 0, 1, 0)

        # Update the received codeword
        y = np.mod(np.dot(G, x_hat) + y, 2)

    decoded_message = ''.join(map(str, x_hat.astype(int)))
    return decoded_message

def main():
    # Example usage
    received_codeword = "1101001"  # Received codeword with possible errors
    decoded_message = ldpc_decode(received_codeword)

    print(f"Received codeword: {received_codeword}")
    print(f"Decoded message: {decoded_message}")

if __name__ == "__main__":
    main()


Received codeword: 1101001
Decoded message: 1101


/var/folders/71/hry96qw555j_rlvg3tfgrh180000gn/T/ipykernel_29804/2802857671.py:38: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  llrs[j] = 2 * y[j] / np.tanh(np.sum(np.arctanh(np.tanh(y[i] / 2)) for i in connected_checks))
